## 4. Report Generation

The final phase of SDMT involves aggregating evidence, validating the metrics reflected by the evidence we collected, and displaying this information in a report.

#### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces.

In [ ]:
import os

from mlte.session import set_context, set_store

store_path = os.path.join(os.getcwd(), "store")

set_context("IrisClassifier", "0.0.1")
set_store(f"local://{store_path}")

### Validate `Evidence` and get an updated `TestResults` with `Result`s

Now that we have our `TestSuite` ready and we have enough evidence, we create a `TestSuiteValidator` with our TestSuite, and add all the `Evidence`s we have. With that we can validate our test cases and generate an output `TestResults`, with the validation results.

In [ ]:
from mlte.validation.test_suite_validator import TestSuiteValidator

# Load validator for default TestSuite id
test_suite_validator = TestSuiteValidator()

# Load all Evidence and validate TestCases
test_results = test_suite_validator.load_and_validate()

# We want to see the validation results in the Notebook, regardless of them being saved.
test_results.print_results()

# TestResults also supports persistence
test_results.save(force=True)

If we want to, we can review test results that could not be automatically validated and require human supervision, and decide whether they pass or not.

In [ ]:
# Show the results that have been marked as informational that could not be validated.
test_results.print_results(result_type="Info")

In [ ]:
# Review the evidence we have for this case.
from IPython import display
from mlte.evidence.types.image import Image

case_id = "evidence.class distribution"
case_image = Image.load(case_id)
display.display(display.Image(data=case_image.image))

Assuming we decided visually that it passed the expectations, we can mark it as such.

In [ ]:
from mlte.results.result import Success

test_results.convert_result(
    test_case_id="class distribution",
    result_type=Success,
    message="Visual inspection confirms distribution is above 1.2%",
)

test_results.print_results()
test_results.save(force=True)

#### Generate a Report

The final step of SDMT involves the generation of a report to communicate the results of model evaluation.

In [ ]:
from mlte.report.artifact import (
    Report,
    CommentDescriptor,
)

# Create a report with the default NegotiationCard, TestSuite and TestResults in this store.
report = Report(
    comments=[
        CommentDescriptor(
            content="This model should not be used for nefarious purposes."
        )
    ],
)

report.save(force=True, parents=True)